!pip install weaviate-client openai tiktoken langchain sentence-transformers transformers > /dev/null

### Objective: 

Understand how the Schema and Querying works in Weviate


The schema is where you define for Weaviate:

Classes
Properties within each Class
Data types for Properties
Graph links between Classes
Vectorizer module (for each Class or Property)

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.vectorstores import Weaviate

In [2]:
def get_text_splits(text_file):
  """Function takes in the text data and returns the  
  splits so for further processing can be done."""
  with open(text_file,'r') as txt:
    data = txt.read()

  textSplit = RecursiveCharacterTextSplitter(chunk_size=150,
                                             chunk_overlap=15,
                                             length_function=len)
  doc_list = textSplit.split_text(data)
  return doc_list

In [52]:
version: '3.4'
services:
  weaviate:
    image: semitechnologies/weaviate:1.18.4
    ports:
    - 8080:8080
    restart: on-failure:0
    environment:
      QUERY_DEFAULTS_LIMIT: 25
      AUTHENTICATION_ANONYMOUS_ACCESS_ENABLED: 'true'
      PERSISTENCE_DATA_PATH: '/var/lib/weaviate'
      DEFAULT_VECTORIZER_MODULE: 'none'
      ENABLE_MODULES: 'text2vec-huggingface'
      CLUSTER_HOSTNAME: 'node1'

SyntaxError: invalid syntax (1442739967.py, line 2)

In [97]:
import weaviate
client = weaviate.Client(
  url="http://localhost:8080",
  additional_headers={
        "X-HuggingFace-Api-Key": ""
    }
)

In [20]:
client.schema.delete_all()

The schema defines schema-level settings (e.g. the vectorizer module), the properties within each class (name, type, description, settings), possible graph links between data objects (cross-references), and the vectorizer module (if any) to be used for the class.

Weaviate class is like collection:

- Is always written with a capital letter first. This is to distinguish them from generic names for cross-referencing.

- Constitutes a distinct vector space. A search in Weaviate is always restricted to a class.

- Can have its own vectorizer. (e.g. one class can have a text2vec-openai vectorizer, and another might have multi2vec-clip vectorizer, or none if you do not intend on using a vectorizer).

- Has property values, where each property specifies the data type to store.


Couple of interesting links:

https://weaviate.io/developers/weaviate/config-refs/datatypes

https://weaviate.io/developers/weaviate/tutorials/schema

https://weaviate.io/developers/weaviate/configuration/schema-configuration

https://weaviate.io/developers/weaviate/configuration/modules

https://weaviate.io/developers/weaviate/configuration/indexes

The first vector index Weaviate supports is HNSW, which is also the default vector index type. Typical for HNSW is that this index type is super fast at query time, but more costly when in the building process (adding data with vectors). I

In [7]:
schema = {
    "classes": [
        {
            "class": "Paragraph",
            "description": "A written paragraph",
         "moduleConfig": {
        "text2vec-huggingface": {
          "model": "sentence-transformers/all-MiniLM-L6-v2",
          "options": {
            "waitForModel": True,
            "useGPU": False,
            "useCache": True
            }
          }
        },
            "properties": [
                {
                    "dataType": ["text"],
                    "description": "The content of the paragraph",
                    "moduleConfig": {
                        "text2vec-huggingface": {
                          "skip": False,
                          "vectorizePropertyName": False
                        }
                      },
                    "name": "content",
                },
            ],
         "vectorizer":"text2vec-huggingface"
        },
    ]
}

In [21]:
client.schema.create(schema)

In [12]:
mail_docs = get_text_splits("mail_collector.txt")

In [24]:
len(mail_docs)

50

In [22]:
import time
# Configure a batch process
with client.batch as batch:
    batch.batch_size=5
    for i, d in enumerate(mail_docs):
        properties = {
            "content": d,
        }
        time.sleep(5)
        client.batch.add_data_object(properties, "Paragraph")

In [37]:
def load_docs(class_name, doc_list):
    # Configure a batch process
    with client.batch as batch:
        batch.batch_size=5
        batch = 0
        for i, d in enumerate(doc_list):
            print(f"processing batch {batch}")
            properties = {
                "content": d,
            }
            time.sleep(5)
            
            batch = batch + 1
            
            client.batch.add_data_object(properties, class_name)

In [100]:
# maximum number of objects returned is 100.
# by default get() method will return arbitrary number of obj
len(client.data_object.get(limit=50,
                           with_vector=True)['objects'])

50

In [104]:
client.data_object.get(limit=1,class_name="Vim_texts",
                           with_vector=False)['objects']

[{'class': 'Vim_texts',
  'creationTimeUnix': 1682749756012,
  'id': '28eff942-57f2-4229-8af5-86c9f60f2357',
  'lastUpdateTimeUnix': 1682749756012,
  'properties': {'content': 'The following are related to the activities on splits'},
  'vectorWeights': None}]

In [25]:
nearText = {"concepts": ["Photograph"]}

In [26]:
result = (
    client.query
    .get("Paragraph", ["content"])
    .with_near_text(nearText)
    .with_limit(2)
    .do()
)

In [27]:
import json
print(json.dumps(result, indent=4))

{
    "data": {
        "Get": {
            "Paragraph": [
                {
                    "content": "2023-04-26 10:44:41+05:30Was this a lucky shot? Although many amazing photographs are taken by someone who just happened\u00c2\u00a0to be in the right place at"
                },
                {
                    "content": "that it took many hours of exposure with a telescope in Seven Persons, Alberta , Canada to create the featured image.April 24, 2023via NASA"
                }
            ]
        }
    }
}


In [24]:
vectorstore = Weaviate(client, 
                       "Paragraph", 
                       "content")

In [ ]:
vectorstore.similarity_search("Photograph")

### Starting the 2nd collection

In [45]:
schema_2 = {
    "class":"Vim_texts",
    "description":"Holds the information regarding vim tutorial",
    "properties":[
        {
            "dataType":["text"],
            "description":"Text Snippet",
            "name":"content"
        }
    ],
    "vectorizer":"text2vec-huggingface"
}

In [44]:
client.schema.delete_class("vim_texts")

In [46]:
client.schema.create_class(schema_class=schema_2)

In [47]:
vim_docs = get_text_splits("vim_play.txt")

In [48]:
load_docs(class_name="Vim_texts",
         doc_list=vim_docs)

processing batch 0
processing batch 1
processing batch 2
processing batch 3
processing batch 4
processing batch 5
processing batch 6
processing batch 7
processing batch 8
processing batch 9
processing batch 10
processing batch 11
processing batch 12
processing batch 13
processing batch 14
processing batch 15
processing batch 16
processing batch 17
processing batch 18


In [50]:
#this is langchain Weviate Object
vim_store = Weaviate(client, 
                       "Vim_texts", 
                       "content")

In [51]:
vim_store.similarity_search("vim tutorial")

[Document(page_content='the activities the command can be referred below', metadata={}),
 Document(page_content='The document will list the various activities at the beginning. The same\nhas to be practiced on the file of your choice. If there is doubt on', metadata={}),
 Document(page_content='1) Split the working Vim window into multiple window by splitting vertically / horizontally.', metadata={}),
 Document(page_content='5) Do the above for entire document, for the current line\n6) Execute the command on last line of the file, current line to last line', metadata={})]

### Learning about the queries

### Simple get method

It has additional parameters

In [55]:
client.data_object.get(limit=1,
                           with_vector=True)['objects']

[{'class': 'Vim_texts',
  'creationTimeUnix': 1682749756012,
  'id': '28eff942-57f2-4229-8af5-86c9f60f2357',
  'lastUpdateTimeUnix': 1682749756012,
  'properties': {'content': 'The following are related to the activities on splits'},
  'vector': [0.38370237,
   -0.036440857,
   -0.25022212,
   -0.04745609,
   0.36459142,
   0.1071606,
   0.5608019,
   0.26321998,
   -0.49892408,
   -0.31748593,
   -0.12759857,
   0.4547883,
   -0.2343596,
   0.22548397,
   0.2964472,
   0.81867707,
   0.4351086,
   0.10718872,
   0.0481725,
   -0.020138578,
   -0.010181775,
   0.73345673,
   0.23478423,
   -0.039970838,
   0.16300847,
   -0.29260895,
   0.053480633,
   0.09042403,
   -0.3456726,
   0.00426801,
   -0.10726686,
   -0.14058897,
   0.39192826,
   -0.05778493,
   -0.2677755,
   -0.46546537,
   -0.29990825,
   -0.21885294,
   0.1570662,
   -0.20792079,
   -0.04797629,
   -0.4068968,
   -0.12299479,
   0.121602334,
   0.1223249,
   -0.2110772,
   -1.1602104,
   0.44173545,
   -0.30196202,
   

### Get with nearText

We need some new data to show the capabilities of the 
vector store. Lets work with a CSV file

In [56]:
with open('space_shortened.csv') as csv:
    data = csv.readlines()

In [59]:
data[0].split(',')

['PassengerId',
 'HomePlanet',
 'CryoSleep',
 'Cabin',
 'Destination',
 'Age',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'Name',
 'Transported\n']

In [72]:
schema3= {
    "class":"Spacefarer",
    "description":"Holds the information regarding vim tutorial",
    "properties":[
        {
            "dataType":["text"],
            "description":"Passenger Identification",
            "name":"passengerid"
        },
        {
            "dataType":["text"],
            "description":"Originating Planet",
            "name":"homePlanet"
        },
        {
            "dataType":["text"],
            "description":"Rest of the Info",
            "name":"info"
        }
    ],
    "vectorizer":"text2vec-huggingface"
}

In [73]:
client.schema.create_class(schema_class=schema3)

In [64]:
data[0].split(',')[2:]

['CryoSleep',
 'Cabin',
 'Destination',
 'Age',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'Name',
 'Transported\n']

In [74]:
# Configure a batch process
with client.batch as batch:
    batch.batch_size=10
    for i, d in enumerate(data):
        properties = {
            "passengerid": d.split(',')[0],
            "homePlanet": d.split(',')[1],
            "info": "".join(d.split(',')[2:]),
        }

        client.batch.add_data_object(properties, "Spacefarer")

In [109]:
nearText = {"concepts": ["Trappist"]}

result = (
    client.query
    .get("Vim_texts", ["content"])
    .with_near_text(nearText) #this method conv text to vector
    .with_limit(2)
    .with_additional(["certainty"])
    .do()
)


In [110]:
result

{'data': {'Get': {'Vim_texts': [{'_additional': {'certainty': 0.9287362694740295},
     'content': '1) Split the working Vim window into multiple window by splitting vertically / horizontally.'},
    {'_additional': {'certainty': 0.9285731613636017},
     'content': '2) Insert the output of your any of the config files in to the current document. Ensure\nyou try out this insert in differnt locations'}]}}}

In [87]:
nearText = {"concepts": ["Trappist"]}

result_certain = (
    client.query
    .get("Spacefarer", ["passengerid", "homePlanet", "info"])
    .with_near_text(nearText) #this method conv text to vector
    .with_limit(2)
    .with_additional(["certainty"])
    .do()
)

In [88]:
result_certain

{'data': {'Get': {'Spacefarer': [{'_additional': {'certainty': 0.9340217709541321},
     'homePlanet': 'Earth',
     'info': 'TrueG/0/STRAPPIST-1e28.0False0.00.00.00.0Candra JacostaffeyTrue\n',
     'passengerid': '0006_02'},
    {'_additional': {'certainty': 0.9339374601840973},
     'homePlanet': 'Earth',
     'info': 'FalseG/1/STRAPPIST-1e48.0False719.01.065.00.024.0Reney BakettonFalse\n',
     'passengerid': '0010_01'}]}}}

### Get with nearVector

In [84]:
client.data_object.get(limit=1,class_name='Spacefarer',
                           with_vector=True)['objects']

[{'class': 'Spacefarer',
  'creationTimeUnix': 1682753419304,
  'id': '00516295-0124-4fc8-80ef-d32dabad4862',
  'lastUpdateTimeUnix': 1682753419304,
  'properties': {'homePlanet': 'Europa',
   'info': 'TrueB/1/PTRAPPIST-1e34.0False0.00.00.00.0Altardr FlaticTrue\n',
   'passengerid': '0008_02'},
  'vector': [0.008160877,
   0.13883571,
   0.46457282,
   -0.26291898,
   0.050365325,
   0.09144319,
   -0.39272013,
   -0.06835353,
   -0.10857654,
   -0.26426807,
   -0.35894498,
   0.22989626,
   -0.0014217547,
   0.4197113,
   0.18912068,
   0.22119713,
   0.46204984,
   0.2152711,
   -0.0007762814,
   -0.22023958,
   0.49010757,
   0.42953542,
   0.14873639,
   -0.05536957,
   -0.28628102,
   -0.28163576,
   -0.23196866,
   0.18105206,
   -0.4160942,
   0.19039364,
   0.08576048,
   -0.07451487,
   -0.3367576,
   -0.20034732,
   0.12579969,
   0.027717752,
   -0.29418904,
   -0.20546675,
   -0.013221663,
   0.042040206,
   0.25821114,
   -0.20239449,
   -0.14662142,
   0.29287228,
   -0.0

In [111]:

result = (
    client.query
    .get("Spacefarer", ["info", "homePlanet","passengerid"])
    .with_near_vector({
        "vector": [0.008160877,
   0.13883571,
   0.46457282,
   -0.26291898,
   0.050365325,
   0.09144319,
   -0.39272013,
   -0.06835353,
   -0.10857654,
   -0.26426807,
   -0.35894498,
   0.22989626,
   -0.0014217547,
   0.4197113,
   0.18912068,
   0.22119713,
   0.46204984,
   0.2152711,
   -0.0007762814,
   -0.22023958,
   0.49010757,
   0.42953542,
   0.14873639,
   -0.05536957,
   -0.28628102,
   -0.28163576,
   -0.23196866,
   0.18105206,
   -0.4160942,
   0.19039364,
   0.08576048,
   -0.07451487,
   -0.3367576,
   -0.20034732,
   0.12579969,
   0.027717752,
   -0.29418904,
   -0.20546675,
   -0.013221663,
   0.042040206,
   0.25821114,
   -0.20239449,
   -0.14662142,
   0.29287228,
   -0.08606034,
   -0.095831305,
   -1.062667,
   0.6950825,
   -0.19287816,
   -0.030068932,
   -0.35192686,
   -0.025985414,
   0.2853342,
   0.50007015,
   -0.0984601,
   0.36388808,
   -0.13351816,
   -0.3564797,
   0.1298961,
   -0.14889574,
   0.5852991,
   0.38425028,
   0.09159476,
   -0.108271286,
   -0.39134538,
   -0.016606815,
   0.1960926,
   0.20297794,
   -0.14730893,
   -0.04126277,
   -0.22795177,
   0.25608566,
   -0.22602162,
   0.10296884,
   0.34407827,
   -0.44401628,
   -0.26365775,
   0.41602966,
   0.227081,
   0.3336095,
   0.36606744,
   -0.33089396,
   0.6021755,
   0.50939935,
   -0.13846812,
   -0.3894231,
   0.08621731,
   -0.35591155,
   -0.3900437,
   0.58113354,
   -0.47697484,
   -0.3028387,
   0.28002158,
   0.0052613188,
   0.47972676,
   -0.4169588,
   -0.19602066,
   -0.11006474,
   0.3077339,
   -0.18115565,
   -0.52871656,
   0.09272504,
   -0.085397884,
   -0.12421829,
   -0.14932245,
   -0.22933578,
   -0.020490404,
   0.03194552,
   -0.05388184,
   0.374174,
   0.16594629,
   -0.11949051,
   -0.10178553,
   -0.28883424,
   -0.044772517,
   0.4112716,
   0.2399392,
   0.23033597,
   0.03934356,
   -0.11738192,
   0.04543093,
   0.82135844,
   -0.014636763,
   -0.37023067,
   0.38246125,
   0.14716023,
   0.24324551,
   0.21280876,
   -0.4301391,
   -0.47323197,
   0.17272253,
   0.026517956,
   -0.20385967,
   0.28406614,
   0.04881394,
   -0.028515635,
   0.2541704,
   0.089107186,
   0.25672778,
   0.11549749,
   -0.32965863,
   -0.39062443,
   -0.12956099,
   0.579154,
   0.3797716,
   0.08761562,
   0.22672461,
   0.004850851,
   -0.7207728,
   -0.25023004,
   -0.37736365,
   -0.10082431,
   0.10232078,
   0.012199979,
   -0.0039995867,
   0.36458433,
   -0.07901586,
   0.1936987,
   -0.042406444,
   0.06974829,
   0.30727628,
   0.14075625,
   -0.08656306,
   -0.07607693,
   0.47218287,
   0.013890188,
   0.2783629,
   0.4135033,
   0.061679676,
   -0.06387616,
   -0.10292062,
   -0.10258984,
   0.5178373,
   -0.24547759,
   -0.22404331,
   0.56043816,
   0.6965914,
   0.103260726,
   0.44464448,
   -0.13076991,
   -1.1881285,
   0.5250074,
   -0.2606409,
   -0.013715736,
   0.32944214,
   -0.21232484,
   0.51059383,
   0.13924749,
   -0.1207924,
   0.33683774,
   -0.42090204,
   -0.4377985,
   0.54912096,
   0.22349286,
   -0.12059986,
   -0.027003068,
   -0.15404391,
   0.011939164,
   0.09654781,
   0.49060792,
   -0.23060036,
   -0.0009880875,
   0.4776227,
   -0.119576804,
   -0.2691866,
   0.5686497,
   -0.063474625,
   -0.12544727,
   0.35783768,
   0.26529458,
   0.015649153,
   -0.19343056,
   0.55091935,
   0.02911473,
   0.33606023,
   -0.1663753,
   0.22188863,
   0.22071308,
   -0.011704071,
   -0.0004370206,
   0.53049874,
   0.47320452,
   0.14610837,
   0.14902258,
   0.004871024,
   1.0653638,
   0.4054953,
   -0.4050751,
   0.13852645,
   -0.074332334,
   -0.09632259,
   -0.26315367,
   0.13057877,
   -0.42116535,
   0.19365129,
   0.29089049,
   -0.11228464,
   -0.30502933,
   -0.05838891,
   -0.40030557,
   0.4950229,
   0.25711745,
   -0.20316258,
   0.50759745,
   -0.074080884,
   0.018976409,
   0.18771888,
   0.025663566,
   -0.014108113,
   -0.10200944,
   -0.53986686,
   0.15470639,
   -0.12832,
   0.0243055,
   0.4734023,
   -0.1934394,
   0.11310986,
   0.0050332765,
   -0.11952143,
   -0.007683368,
   0.4970931,
   -0.021538468,
   -0.08106987,
   -0.0856339,
   -0.09346433,
   -0.42268294,
   -0.35514417,
   0.006506795,
   0.35556543,
   0.17136478,
   0.34824836,
   -0.010155085,
   0.032204974,
   -0.25045884,
   -0.36177447,
   -0.009699752,
   0.22084688,
   0.17813705,
   0.25197908,
   -0.28715047,
   -0.17417908,
   0.33678412,
   -0.082880616,
   0.060251758,
   -0.26724902,
   -0.0705941,
   -0.14909145,
   -0.24707703,
   0.2548459,
   0.1889203,
   -0.12166164,
   0.09350748,
   -0.46845973,
   0.19105369,
   0.1475872,
   0.4140302,
   0.1573492,
   0.20168447,
   -0.14826848,
   0.0379614,
   -0.49829748,
   0.5000516,
   -0.46023595,
   0.035887882,
   0.30104443,
   -0.3168038,
   -0.174651,
   -0.101211354,
   -11.015264,
   0.09191572,
   -0.3842313,
   -0.40833932,
   0.07705922,
   0.38714713,
   0.1633083,
   0.24885565,
   0.42483252,
   -0.1256619,
   0.022382574,
   0.4235811,
   -0.028984463,
   -0.009513362,
   0.4262717,
   -0.040364373,
   -0.29084778,
   0.008584754,
   0.13580145,
   -0.024225695,
   -0.19329748,
   -0.39376527,
   0.21329531,
   0.10155406,
   0.371429,
   0.11099469,
   -0.006536775,
   -0.18852806,
   -0.33325827,
   -0.16205096,
   0.105124235,
   0.059299543,
   -0.16015735,
   0.0018839971,
   0.34128693,
   -0.6585377,
   0.24650468,
   -0.42154646,
   0.11653714,
   0.056491777,
   0.23277092,
   -0.2113797,
   -0.15356731,
   -0.2500637,
   0.16478348,
   -0.15768309,
   -0.06576579,
   0.17279641,
   0.1836975,
   0.41084993,
   -0.20357285,
   0.32261646,
   0.06983724,
   -0.13852271,
   -0.08553089,
   0.19201428,
   -0.40307653,
   0.1569565,
   -0.24931425,
   -0.050012052,
   0.46584636,
   -0.12567772,
   -0.31890777,
   -0.14630157,
   -0.3148906,
   0.16307099,
   -0.26038808,
   -0.5531216,
   0.16745037,
   -0.13941461,
   -0.520145,
   0.21177946,
   0.011941496,
   -2.3496187,
   -0.12781076,
   -0.087911926,
   -0.2900468,
   -0.5823377,
   -0.07361981,
   0.0334418,
   -0.20698066,
   -0.023978649,
   -0.2044804,
   0.3105315,
   -0.0960466,
   -0.027305655,
   0.015577429,
   0.12068475,
   -0.062480032,
   -0.27281493,
   0.6205251,
   -0.8654571,
   -0.25245833,
   -0.05767512,
   0.17367825,
   0.22815093,
   -0.64223075,
   0.10082411,
   0.61268306,
   -0.0025033227,
   0.1693613,
   -0.40144107,
   -0.14110985,
   0.053854417,
   0.11861687,
   -0.14706513,
   -0.030677972,
   0.05473616,
   -0.11916541,
   -0.138043,
   0.44612017,
   0.06985445,
   -0.09619634,
   -0.22902514,
   0.13709185,
   -0.055776276,
   0.16853271,
   0.5287193,
   0.35664064,
   -0.21459575,
   -0.107766606,
   0.12967516,
   -0.42512417,
   0.19056892,
   -0.45119008,
   0.51697564,
   -0.37238598,
   0.18029766,
   0.21914786,
   -0.14512895,
   0.02284549,
   0.12151498,
   -0.017147236,
   -0.12055369,
   0.34553936,
   0.09268821,
   -0.34326175,
   -0.6908205,
   0.19797334,
   0.032775223,
   0.23763189,
   -0.027395511,
   0.024742277,
   -0.11487274,
   0.19300756,
   0.13325426,
   0.3489062,
   -0.33034623,
   -0.20808133,
   0.29405844,
   0.122177884,
   -0.52931756,
   0.4486384,
   0.009135126,
   -0.18167736,
   0.18403068,
   0.5326617,
   0.2127197,
   -0.43166274,
   0.13366275,
   0.4778099,
   -0.4205399,
   -0.5195915,
   0.04778018,
   0.32639483,
   0.31002054,
   0.04462336,
   -0.26411012,
   0.10631967,
   -0.14331578,
   -0.049571875,
   -0.074755386,
   0.0054414417,
   0.33947602,
   0.16858365,
   0.055250823,
   -0.11742325,
   -0.10351422,
   0.16908509,
   0.09567665,
   -0.012282666,
   0.21541725,
   0.18067041,
   -0.089670494,
   -0.14256865,
   -0.2046576,
   -0.013927072,
   -0.36434558,
   -0.03603621,
   -0.11706719,
   0.19790678,
   0.6024332,
   -0.073971815,
   0.10928726,
   -0.09734234,
   -0.74704355,
   -0.14233263,
   0.57157636,
   0.09623336,
   0.41187227,
   -0.40439242,
   -0.1630651,
   -0.32728547,
   -0.08915789,
   0.26007497,
   -0.14401893,
   0.4921589,
   -0.10014284,
   0.08353439,
   0.030141419,
   -0.24271978,
   -0.6868821,
   -0.29091075,
   0.42126572,
   -0.13089766,
   0.36118105,
   -0.023873793,
   -0.15790975,
   -0.4470271,
   0.013560564,
   0.1813879,
   0.060562894,
   -0.093056895,
   -0.07619628,
   0.2712386,
   0.3685212,
   0.11946333,
   -0.42687,
   0.031113941,
   -0.0690942,
   -0.048542164,
   -0.25703144,
   0.16874775,
   0.380283,
   -0.26805925,
   -0.06669061,
   -0.6738544,
   -0.30395675,
   0.0072268923,
   0.28064978,
   -0.014231416,
   -0.08137358,
   0.19222207,
   -0.31230834,
   -0.23616056,
   0.28203306,
   0.1747266,
   0.17423894,
   0.19257036,
   -0.20168534,
   -0.04469389,
   -0.4071956,
   -0.13786906,
   -0.08465291,
   0.27890965,
   -0.6440251,
   -0.53205186,
   0.17661928,
   -0.3787469,
   -0.15637179,
   -0.10233898,
   -0.027962506,
   -0.41659304,
   -0.100200675,
   0.35340288,
   -0.0601297,
   0.15229888,
   -0.4286706,
   0.3789958,
   -0.017103689,
   0.661394,
   0.20552093,
   0.2313479,
   0.22093478,
   0.112226464,
   0.10176368,
   -0.02897533,
   -0.030694105,
   0.4913211,
   0.17970563,
   -0.2133105,
   0.13946997,
   -0.34985542,
   0.11663311,
   -0.16079965,
   -0.2215149,
   -0.050232854,
   -0.08199573,
   -0.23593928,
   0.12413323,
   0.12683234,
   0.122601695,
   -0.4426508,
   -0.105834335,
   -0.03266425,
   0.04316636,
   0.38862184,
   0.30099252,
   -0.22875343,
   0.43128946,
   -0.50072217,
   0.030426778,
   -0.09393061,
   -0.32194826,
   0.17025818,
   0.10931613,
   0.24622744,
   -0.5703292,
   -0.06402667,
   0.07667101,
   0.11307537,
   -0.34559572,
   0.20628987,
   0.043170407,
   0.07864652,
   -0.1622641,
   0.38536608,
   -0.2845537,
   -0.4137633,
   0.19097896,
   0.27988586,
   0.07338988,
   0.27831385,
   0.13295726,
   0.47613716,
   0.16287312,
   -0.035791565,
   0.42680523,
   -0.30069253,
   0.3181979,
   0.4860481,
   0.19311251,
   0.5002956,
   0.43667924,
   -0.34457028,
   0.27809867,
   0.09781528,
   -0.364824,
   -0.52396435,
   0.14361335,
   0.06002282,
   0.29914397,
   -0.6823961,
   -0.048237786,
   0.40207767,
   0.17214164,
   0.061487567,
   -0.303644,
   0.13544804,
   0.30639666,
   -0.23949566,
   0.16951278,
   -0.057289492,
   -0.050099786,
   0.14532378,
   -0.15803127,
   0.1836546,
   -0.3021684,
   -0.78491956,
   -0.033373043,
   0.04835292,
   -0.07576623,
   0.45808455,
   -0.074631624,
   -0.4199278,
   0.02293988,
   0.035873715,
   -0.098070376,
   0.3436448,
   -0.13613589,
   0.43258944,
   -0.07945802,
   -0.050351594,
   0.15589996,
   0.21291399,
   -0.33654264,
   0.008692318,
   -0.14265962,
   0.13506615,
   0.40154955,
   -0.5045447,
   -0.46881387,
   -0.21555084,
   -0.35550144,
   -0.14888622,
   -0.6637882,
   0.122728154,
   -0.21984659,
   -0.31297866,
   -0.35272685,
   -0.20000592,
   -0.31419608,
   -0.08795013,
   0.35294867,
   0.034337964,
   -0.13573545,
   0.09332047,
   -0.15845613,
   -0.0022304864,
   0.050507214,
   0.50099325,
   0.3206435,
   -0.007117873,
   0.13050623,
   0.06526126,
   0.2547993,
   -0.09358501,
   -0.17834224,
   -0.5832203,
   0.18269558,
   0.08063214,
   -0.08235807,
   -0.120492525,
   0.69289726,
   -0.048694406,
   0.17862052,
   0.52720666,
   -0.07149411,
   0.1664688,
   -0.28383514,
   -0.22938618,
   -0.1666531,
   -0.10597818,
   0.156317,
   -0.38023853,
   0.113479346,
   0.27079192,
   -0.34094417,
   0.33754793,
   0.50448745,
   0.10481549,
   -0.1290099,
   -0.38036874,
   0.28957462,
   -0.13301395,
   -0.42584088,
   -0.19514857,
   -0.23078671,
   -0.16701454,
   -0.3092488,
   0.13460511,
   0.17078029,
   -0.15728882,
   0.4988453,
   0.0488134,
   -0.0939777,
   0.30824968,
   -0.30936262,
   -0.41876507,
   0.35438126,
   0.30788705,
   0.030099066,
   0.16559358,
   -0.5990109,
   0.39253387,
   -0.10575315,
   0.20626788,
   0.09074866,
   0.5029287,
   0.34726536],
        "certainty": 0.7
    })
    .with_limit(2)
    .do()
)

print(json.dumps(result, indent=4))

{
    "data": {
        "Get": {
            "Spacefarer": [
                {
                    "homePlanet": "Europa",
                    "info": "TrueB/1/PTRAPPIST-1e34.0False0.00.00.00.0Altardr FlaticTrue\n",
                    "passengerid": "0008_02"
                },
                {
                    "homePlanet": "Europa",
                    "info": "FalseB/0/PTRAPPIST-1e39.0False0.00.00.00.00.0Maham OfracculyFalse\n",
                    "passengerid": "0001_01"
                }
            ]
        }
    }
}


In [119]:
where_filter = {
    "path": ["homePlanet"],
    "operator": "Equal",
    "valueText": "Earth",
}

In [116]:
nearText = {"concepts": ["Trappist"]}

result_earth = (
    client.query
    .get("Spacefarer", ["passengerid", "homePlanet", "info"])
    .with_near_text(nearText) #this method conv text to vector
    .with_where(where_filter)
    .do()
)

In [117]:
print(json.dumps(result_earth, indent=4))

{
    "data": {
        "Get": {
            "Spacefarer": [
                {
                    "homePlanet": "Europa",
                    "info": "FalseA/0/STRAPPIST-1e58.0True43.03576.00.06715.049.0Altark SusentFalse\n",
                    "passengerid": "0003_01"
                },
                {
                    "homePlanet": "Europa",
                    "info": "FalseA/0/STRAPPIST-1e33.0False0.01283.0371.03329.0193.0Solam SusentFalse\n",
                    "passengerid": "0003_02"
                },
                {
                    "homePlanet": "HomePlanet",
                    "info": "CryoSleepCabinDestinationAgeVIPRoomServiceFoodCourtShoppingMallSpaVRDeckNameTransported\n",
                    "passengerid": "PassengerId"
                },
                {
                    "homePlanet": "Mars",
                    "info": "FalseF/1/PTRAPPIST-1e32.0False73.00.01123.00.0113.0Berers BarneTrue\n",
                    "passengerid": "0009_01"
                

In [95]:
result_agg = (
    client.query
    .aggregate("Spacefarer")
    .with_fields("meta {count}")
    .do()
)

print(json.dumps(result_agg, indent=4))

{
    "data": {
        "Aggregate": {
            "Spacefarer": [
                {
                    "meta": {
                        "count": 17
                    }
                }
            ]
        }
    }
}


In [120]:
result_agg = (
    client.query
    .aggregate("Spacefarer")
    .with_fields("meta {count}")
    .with_where(where_filter)
    .do()
)

print(json.dumps(result_agg, indent=4))

{
    "data": {
        "Aggregate": {
            "Spacefarer": [
                {
                    "meta": {
                        "count": 9
                    }
                }
            ]
        }
    }
}
